In [0]:
def get_dataset(num_classes, rank=0, size=1):
  from tensorflow import keras
  
  (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data('MNIST-data-%d' % rank)
  x_train = x_train[rank::size]
  y_train = y_train[rank::size]
  x_test = x_test[rank::size]
  y_test = y_test[rank::size]
  x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
  x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)
  return (x_train, y_train), (x_test, y_test)

In [0]:
def get_model(num_classes):
  from tensorflow.keras import models
  from tensorflow.keras import layers
  
  model = models.Sequential()
  model.add(layers.Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=(28, 28, 1)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes, activation='softmax'))
  return model

In [0]:
# Specify training parameters
batch_size = 128
epochs = 2
num_classes = 10

def train(learning_rate=0.1, batch_size=64, epochs=10, num_classes=10):
  from tensorflow import keras
  
  (x_train, y_train), (x_test, y_test) = get_dataset(num_classes)
  model = get_model(num_classes)

  optimizer = keras.optimizers.Adadelta(lr=learning_rate)

  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=2,
            validation_data=(x_test, y_test))
  return model

In [0]:
model = train(learning_rate=0.1, batch_size=64, epochs=10, num_classes=10)

Epoch 1/2
469/469 - 105s - loss: 0.6112 - accuracy: 0.8137 - val_loss: 0.2201 - val_accuracy: 0.9363
Epoch 2/2
469/469 - 105s - loss: 0.3151 - accuracy: 0.9041 - val_loss: 0.1657 - val_accuracy: 0.9518

In [0]:
_, (x_test, y_test) = get_dataset(num_classes)
loss, accuracy = model.evaluate(x_test, y_test, batch_size=128)
print("loss:", loss)
print("accuracy:", accuracy)

1/79 [..............................] - ETA: 5s - loss: 0.0892 - accuracy: 0.9922 3/79 [>.............................] - ETA: 3s - loss: 0.1369 - accuracy: 0.9609 5/79 [>.............................] - ETA: 3s - loss: 0.1697 - accuracy: 0.9438 7/79 [=>............................] - ETA: 3s - loss: 0.1650 - accuracy: 0.9453 8/79 [==>...........................] - ETA: 3s - loss: 0.1735 - accuracy: 0.945310/79 [==>...........................] - ETA: 3s - loss: 0.1949 - accuracy: 0.938312/79 [===>..........................] - ETA: 3s - loss: 0.2032 - accuracy: 0.939514/79 [====>.........................] - ETA: 3s - loss: 0.2135 - accuracy: 0.936916/79 [=====>........................] - ETA: 2s - loss: 0.2156 - accuracy: 0.936018/79 [=====>........................] - ETA: 2s - loss: 0.2261 - accuracy: 0.934520/79 [======>.......................] - ETA: 2s - loss: 0.2217 - accuracy: 0.934822/79 [=======>......................] - ETA: 2s - loss: 0.2183 - accuracy: 0.936124/79 [========>.....................] - ETA: 2s - loss: 0.2127 - accuracy: 0.939126/79 [========>.....................] - ETA: 2s - loss: 0.2107 - accuracy: 0.939628/79 [=========>....................] - ETA: 2s - loss: 0.2080 - accuracy: 0.940630/79 [==========>...................] - ETA: 2s - loss: 0.2137 - accuracy: 0.938332/79 [===========>..................] - ETA: 2s - loss: 0.2189 - accuracy: 0.936034/79 [===========>..................] - ETA: 2s - loss: 0.2204 - accuracy: 0.935236/79 [============>.................] - ETA: 1s - loss: 0.2192 - accuracy: 0.935538/79 [=============>................] - ETA: 1s - loss: 0.2168 - accuracy: 0.936740/79 [==============>...............] - ETA: 1s - loss: 0.2141 - accuracy: 0.937542/79 [==============>...............] - ETA: 1s - loss: 0.2076 - accuracy: 0.939544/79 [===============>..............] - ETA: 1s - loss: 0.2004 - accuracy: 0.941646/79 [================>.............] - ETA: 1s - loss: 0.1982 - accuracy: 0.942448/79 [=================>............] - 

In [0]:
import os
import time
checkpoint_dir = '/dbfs/ml/MNISTDemo/train/{}/'.format(time.time())
os.makedirs(checkpoint_dir)
print(checkpoint_dir)

/dbfs/ml/MNISTDemo/train/1622452995.1035428/

The following cell shows how to modify the single-node code of the previously defined `train()` function to take advantage of distributed training.

In [0]:
def train_hvd(learning_rate, batch_size, checkpoint_path):
  
  from tensorflow.keras import backend as K
  from tensorflow.keras.models import Sequential
  import tensorflow as tf
  from tensorflow import keras
  import horovod.tensorflow.keras as hvd

  hvd.init()

  gpus = tf.config.experimental.list_physical_devices('GPU')
  for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
  if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

  (x_train, y_train), (x_test, y_test) = get_dataset(num_classes, hvd.rank(), hvd.size())
  model = get_model(num_classes)

  optimizer = keras.optimizers.Adadelta(lr=learning_rate * hvd.size())

  optimizer = hvd.DistributedOptimizer(optimizer)

  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  callbacks = [
      hvd.callbacks.BroadcastGlobalVariablesCallback(0),
  ]

  if hvd.rank() == 0:
      callbacks.append(keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only = True))

  model.fit(x_train, y_train,
            batch_size=batch_size,
            callbacks=callbacks,
            epochs=epochs,
            verbose=2,
            validation_data=(x_test, y_test))

In [0]:
from sparkdl import HorovodRunner

checkpoint_path = checkpoint_dir + '/checkpoint-{epoch}.ckpt'
learning_rate = 0.1
hr = HorovodRunner(np=2)
hr.run(train_hvd, checkpoint_path=checkpoint_path, learning_rate=learning_rate)

HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'log_callback_only', like
`HorovodRunner(np=2, driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'num_classes', 'batch_size', 'epochs', 'get_model', 'get_dataset'}.
The pickled object size is 3811 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.

In [0]:
import tensorflow as tf

hvd_model = get_model(num_classes)
hvd_model.compile(optimizer=tf.keras.optimizers.Adadelta(lr=learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
hvd_model.load_weights(tf.train.latest_checkpoint(os.path.dirname(checkpoint_path)))

Out[9]: <tensorflow.python.training.tracking.util.CheckpointLoadStatus at 0x7f235c7b2640>

In [0]:
num_classes=10
_, (x_test, y_test) = get_dataset(num_classes)
loss, accuracy = hvd_model.evaluate(x_test, y_test, batch_size=128)
print("loaded model loss and accuracy:", loss, accuracy)

1/79 [..............................] - ETA: 21s - loss: 0.0772 - accuracy: 0.9844 3/79 [>.............................] - ETA: 3s - loss: 0.0991 - accuracy: 0.9770  5/79 [>.............................] - ETA: 2s - loss: 0.1151 - accuracy: 0.9687 7/79 [=>............................] - ETA: 2s - loss: 0.1227 - accuracy: 0.9639 9/79 [==>...........................] - ETA: 2s - loss: 0.1277 - accuracy: 0.960811/79 [===>..........................] - ETA: 2s - loss: 0.1351 - accuracy: 0.957713/79 [===>..........................] - ETA: 2s - loss: 0.1417 - accuracy: 0.955215/79 [====>.........................] - ETA: 2s - loss: 0.1472 - accuracy: 0.953617/79 [=====>........................] - ETA: 2s - loss: 0.1520 - accuracy: 0.952119/79 [======>.......................] - ETA: 2s - loss: 0.1566 - accuracy: 0.950721/79 [======>.......................] - ETA: 2s - loss: 0.1598 - accuracy: 0.949823/79 [=======>......................] - ETA: 2s - loss: 0.1621 - accuracy: 0.949225/79 [========>.....................] - ETA: 2s - loss: 0.1639 - accuracy: 0.948727/79 [=========>....................] - ETA: 2s - loss: 0.1650 - accuracy: 0.948529/79 [==========>...................] - ETA: 2s - loss: 0.1659 - accuracy: 0.948331/79 [==========>...................] - ETA: 2s - loss: 0.1671 - accuracy: 0.948133/79 [===========>..................] - ETA: 1s - loss: 0.1682 - accuracy: 0.947835/79 [============>.................] - ETA: 1s - loss: 0.1692 - accuracy: 0.947637/79 [=============>................] - ETA: 1s - loss: 0.1700 - accuracy: 0.947439/79 [=============>................] - ETA: 1s - loss: 0.1708 - accuracy: 0.947240/79 [==============>...............] - ETA: 1s - loss: 0.1711 - accuracy: 0.947142/79 [==============>...............] - ETA: 1s - loss: 0.1715 - accuracy: 0.947044/79 [===============>..............] - ETA: 1s - loss: 0.1715 - accuracy: 0.947046/79 [================>.............] - ETA: 1s - loss: 0.1715 - accuracy: 0.947148/79 [=================>............]

In [0]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sparkdl import HorovodRunner
def wrap_train(params):
  hr = HorovodRunner(np=2)
  loss, acc = hr.run(train_hvd,
                     learning_rate=params['learning_rate'],
                     batch_size=params['batch_size'],
                     checkpoint_dir=checkpoint_dir)
  return {'loss': loss, 'status': STATUS_OK}

In [0]:
import numpy as np
space = {
  'learning_rate': hp.loguniform('learning_rate', np.log(1e-4), np.log(1e-1)),
  'batch_size': hp.choice('batch_size', [32, 64, 128]),
}

In [0]:
algo=tpe.suggest

best_param = fmin(
  fn=train,
  space=space,
  algo=algo,
  max_evals=8,
  return_argmin=False,
)

0%| | 0/8 [00:00<?, ?trial/s, best loss=?]HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'log_callback_only', like
`HorovodRunner(np=2, driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'num_classes', 'epochs', 'os', 'get_model', 'get_dataset'}.
The pickled object size is 4162 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.
 12%|█▎ | 1/8 [02:07<14:50, 127.25s/trial, best loss: 1.8650190830230713]HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'log_callback_only', like
`HorovodRunner(np=2, driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'num_classes', 'epochs', 'os', 'get_model', 'get_dataset'}.
The pickled object size is 4162 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.
 25%|██▌ | 2/8 [04:25<13:03, 130.57s/trial, best loss: 1.8650190830230713]HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'log_callback_only', like
`HorovodRunner(np=2, driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'num_classes', 'epochs', 'os', 'get_model', 'get_dataset'}.
The pickled object size is 4162 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.
 38%|███▊ | 3/8 [07:06<11:37, 139.58s/trial, best loss: 0.30220600962638855]HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'log_callback_only', like
`HorovodRunner(np=2, driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'num_classes', 'epochs', 'os', 'get_model', 'get_dataset'}.
The pickled object size is 4162 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.
 50%|█████ | 4/8 [09:49<09:47, 146.82s/trial, best loss: 0.17444933950901031]HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell o

In [0]:
print(best_param)

{'batch_size': 32, 'learning_rate': 0.02498587538781943}

In [0]:
%pip install spark-tensorflow-distributor

Python interpreter will be restarted.
Requirement already satisfied: spark-tensorflow-distributor in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (0.1.0)
Collecting tensorflow>=2.1.0
 Downloading tensorflow-2.5.0-cp38-cp38-manylinux2010_x86_64.whl (454.4 MB)
Requirement already satisfied: numpy~=1.19.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (1.19.2)
Requirement already satisfied: six~=1.15.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (1.15.0)
Collecting tensorflow-estimator<2.6.0,>=2.5.0rc0
 Downloading tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
Collecting h5py~=3.1.0
 Downloading h5py-3.1.0-cp38-cp38-manylinux1_x86_64.whl (4.4 MB)
Collecting tensorboard~=2.5
 Downloading tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
Requirement already satisfied: wrapt~=1.12.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (1.12.1)
Requirement already satisfied: typing-extensions~=3.7.4 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (3.7.4.3)
Requirement already satisfied: absl-py~=0.10 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (0.11.0)
Collecting keras-nightly~=2.5.0.dev
 Downloading keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
Collecting grpcio~=1.34.0
 Downloading grpcio-1.34.1-cp38-cp38-manylinux2014_x86_64.whl (4.0 MB)
Collecting gast==0.4.0
 Downloading gast-0.4.0-py3-none-any.whl (9.8 kB)
Requirement already satisfied: keras-preprocessing~=1.1.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (1.1.2)
Requirement already satisfied: termcolor~=1.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (1.1.0)
Requirement already satisfied: opt-einsum~=3.3.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (3.3.0)
Requirement already satisfied: protobuf>=3.9.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (3.13.0)
Requirement already satisfied: google-pasta~=0.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (0.2.0)
Requirement already satisfied: wheel~=0.35 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (0.35.1)
Requirement already satisfied: astunparse~=1.6.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (1.6.3)
Requirement already satisfied: flatbuffers~=1.12.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorflow>=2.1.0->spark-tensorflow-distributor) (1.12)
Requirement already satisfied: setuptools>=41.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5c3001d5-a862-4151-bdd5-099d1792d805/lib/python3.8/site-packages (from tensorboard~=2.5->tensorflow>=2.1.0->sp

In [0]:
import tensorflow as tf
NUM_WORKERS = 2
CLUSTER_GPUS = len(tf.config.list_logical_devices('GPU')) * NUM_WORKERS
USE_GPU = CLUSTER_GPUS > 0

In [0]:
def train(BUFFER_SIZE = 5000, BATCH_SIZE = 32, learning_rate=0.005):
  import tensorflow as tf
  import uuid
  def get_datasets():
    (mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data(path=str(uuid.uuid4())+'mnist.npz')
    dataset = tf.data.Dataset.from_tensor_slices((
      tf.cast(mnist_images[..., tf.newaxis] / 255.0, tf.float32),
      tf.cast(mnist_labels, tf.int64))
    )
    dataset = dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return dataset
  
  def get_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax'),
    ])
    model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
      metrics=['accuracy'],
    )
    return model
  
  train_datasets = get_datasets()
  worker_model = get_model()
  options = tf.data.Options()
  options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
  train_datasets = train_datasets.with_options(options)
  worker_model.fit(x=train_datasets, epochs=3, steps_per_epoch=5)

In [0]:
from spark_tensorflow_distributor import MirroredStrategyRunner
runner = MirroredStrategyRunner(num_slots=1, local_mode=True, use_gpu=USE_GPU)
runner.run(train)

Doing CPU training...
MirroredStrategyRunner will run in local mode on the driver node. There would be resource contention if the driver also runs other workloads.
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
 8192/11490434 [..............................] - ETA: 0s 122880/11490434 [..............................] - ETA: 4s 942080/11490434 [=>............................] - ETA: 1s 6471680/11490434 [===============>..............] - ETA: 0s11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/3
1/5 [=====>........................] - ETA: 9s - loss: 2.2895 - accuracy: 0.03125/5 [==============================] - 3s 12ms/step - loss: 2.3100 - accuracy: 0.0375
Epoch 2/3
1/5 [=====>........................] - ETA: 0s - loss: 2.3292 - accuracy: 0.12505/5 [==============================] - 0s 10ms/step - loss: 2.3076 - accuracy: 0.0688
Epoch 3/3
1/5 [=====>........................] - ETA: 0s - loss: 2.3134 - accuracy: 0.06255/5 [==============================] - 0s 12ms/step - loss: 2.2991 - accuracy: 0.0750

In [0]:
NUM_SLOTS = CLUSTER_GPUS if USE_GPU else 4  
runner = MirroredStrategyRunner(num_slots=4, use_gpu=USE_GPU)
runner.run(train)

Doing CPU training...
Will run with 4 Spark tasks.
Distributed training in progress...
View Spark executor stderr logs to inspect training...
Training with 4 slots is complete!

In [0]:
def train_custom_strategy():
  import tensorflow as tf
  strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
    tf.distribute.experimental.CollectiveCommunication.NCCL)
  with strategy.scope():
    import uuid
    BUFFER_SIZE = 10000
    BATCH_SIZE = 32
    def make_datasets():
      (mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data(path=str(uuid.uuid4())+'mnist.npz')
      dataset = tf.data.Dataset.from_tensor_slices((
          tf.cast(mnist_images[..., tf.newaxis] / 255.0,
                  tf.float32),
          tf.cast(mnist_labels, tf.int64))
      )
      dataset = dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
      return dataset

    def get_model():
      model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(138, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax'),
      ])
      model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
        metrics=['accuracy'],
      )
      return model

    train_datasets = make_datasets()
    worker_model = get_model()
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = \
        tf.data.experimental.AutoShardPolicy.DATA
    train_datasets = train_datasets.with_options(options)
    worker_model.fit(x=train_datasets, epochs=3, steps_per_epoch=5)

In [0]:
runner = MirroredStrategyRunner(num_slots=1,
                                use_custom_strategy=True,
                                local_mode=True,
                                use_gpu=USE_GPU)
runner.run(train_custom_strategy)

Doing CPU training...
MirroredStrategyRunner will run in local mode on the driver node. There would be resource contention if the driver also runs other workloads.
WARNING:tensorflow:Enabled NCCL communication but no GPUs detected/specified.
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.NCCL
Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
 8192/11490434 [..............................] - ETA: 0s 4808704/11490434 [===========>..................] - ETA: 0s 6905856/11490434 [=================>............] - ETA: 0s11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/3
1/5 [=====>........................] - ETA: 9s - loss: 2.3512 - accuracy: 0.09385/5 [==============================] - 3s 12ms/step - loss: 2.3500 - accuracy: 0.0500
Epoch 2/3
1/5 [=====>........................] - ETA: 0s - loss: 2.2904 - accuracy: 0.18755/5 [==============================] - 0s 11ms/step - loss: 2.3148 - accuracy: 0.1312
Epoch 3/3
1/5 [=====>........................] - ETA: 0s - loss: 2.3294 - accuracy: 0.09385/5 [==============================] - 0s 12ms/step - loss: 2.3141 - accuracy: 0.1063